In [18]:
!pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
BATCH_SIZE = 512

In [21]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [22]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [26]:
EPOCHS=20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Net().to(DEVICE)
model = model.to(DEVICE)
optimizer = optim.Adam(model.parameters())


for epoch in range(1, EPOCHS + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.597703
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.438407
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.272489

Test set: Average loss: 0.2243, Accuracy: 9335/10000 (93%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.198478
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.223042
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.165552

Test set: Average loss: 0.1129, Accuracy: 9643/10000 (96%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.104594
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.109400
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.122475

Test set: Average loss: 0.0778, Accuracy: 9757/10000 (98%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.084311
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.079413
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.091441

Test set: Average loss: 0.0692, Accuracy: 9787/10000 (98%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.061937
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.052609
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.100486

Test 